In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 2.7MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd 
import numpy as np 

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae 
from sklearn.model_selection import cross_val_score, KFold

import eli5 
from eli5.sklearn import PermutationImportance


In [4]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [6]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0],list): continue
  
  factorized_values = df[feat].factorize()[0]
  
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat+ SUFFIX_CAT] = factorized_values

In [8]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values
  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

## DecisionTree

In [16]:
run_model( DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015792)

## Random Forest

In [17]:

model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(model, cat_feats)

(-18718.657185256638, 64.5424578125788)

## XGBoost

In [19]:
xgb_params = {
    'max_depth':5,
    'n_estimators':50,
    'learning_rate':0.1,
    'seed': 0
}
run_model(xgb.XGBRegressor(**xgb_params), cat_feats)

[18:34:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:35:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:35:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [20]:
m = xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0)
m.fit(X,y)

imp=PermutationImportance(m, random_state=0).fit(X,y)
eli5.show_weights(imp, feature_names=cat_feats)

[18:38:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1194 ± 0.0031,param_napęd__cat
0.1132 ± 0.0032,param_rok-produkcji__cat
0.1090 ± 0.0025,param_stan__cat
0.0619 ± 0.0025,param_skrzynia-biegów__cat
0.0568 ± 0.0016,param_faktura-vat__cat
0.0489 ± 0.0014,param_moc__cat
0.0273 ± 0.0007,param_marka-pojazdu__cat
0.0242 ± 0.0013,feature_kamera-cofania__cat
0.0212 ± 0.0008,param_typ__cat
0.0174 ± 0.0008,param_pojemność-skokowa__cat


In [0]:
	feats= ['param_napęd__cat',
	'param_rok-produkcji',  #usunieto __cat
	'param_stan__cat',
	'param_skrzynia-biegów__cat',
	'param_faktura-vat__cat',
	'param_moc', #usunieto __cat
	'param_marka-pojazdu__cat',
	'feature_kamera-cofania__cat',
	'param_typ__cat',
	'param_pojemność-skokowa', #usunieto __cat
	'seller_name__cat',
	'feature_wspomaganie-kierownicy__cat',
	'param_model-pojazdu__cat',
	'param_wersja__cat',
	'param_kod-silnika__cat',
	'feature_system-start-stop__cat',
	'feature_asystent-pasa-ruchu__cat',
	'feature_czujniki-parkowania-przednie__cat',
	'feature_łopatki-zmiany-biegów__cat',
	'feature_regulowane-zawieszenie__cat']

In [22]:
len(feats)

20

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))  # edytujemy poszczegolne zmienne tak, zeby miały większą wartość objaśniającą (zmienne ciągłe)
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))

In [38]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[18:57:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:57:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:57:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9716.450230340148, 62.2164408321879)

In [55]:
df['param_pojemność-skokowa'].unique()

array([898,   1,   3,   2,   4, 139,  22,  -1, 500, 998, 156, 298, 999,
       125, 900, 995, 647,   6, 200,   5, 996, 136,  13, 973,  16, 652,
       146, 400,  15,  19, 875,  89, 199,  36,  20,  18, 997,  25, 800,
       122, 650, 988, 300, 350, 124,  14, 600, 993, 214,  12, 954,  24,
       190, 505, 697, 270, 191,  30, 899, 847, 159, 796,  50,  29, 599,
       198, 245,  99, 989, 698, 192, 528, 700, 196, 220, 635, 225, 480,
       120,   7, 189, 535, 268, 180, 197, 479, 986, 160, 119,  17, 523,
       179, 249, 100, 223, 279, 130,   8,  27, 280, 990, 319,  40,  42,
       296, 133, 799, 659,  79, 149, 110,  10, 651, 115, 320, 440, 971,
        92, 187, 330, 175, 704, 598,  11, 239, 195, 750, 166, 896, 992,
       934, 235, 143,  38, 140, 138, 297, 842, 890, 450,   9, 699, 903,
        28, 240, 299, 217, 499, 430, 269,  32, 178, 277, 594,  21,  41,
       578, 158, 292, 174, 150,  35, 918, 466, 141, 168,  48,  44, 142])

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ','')))

In [61]:
run_model(xgb.XGBRegressor(**xgb_params), feats)

[19:17:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:17:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:17:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9719.350303828369, 66.39777001545997)

In [62]:
!git init
!git add .

Reinitialized existing Git repository in /content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car/.git/


In [65]:
!git config --global user.email 'konrad.wasikiewicz@gmail.com'
!git config --global user.name 'KonradWasikiewicz'
!git commit

On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean


In [68]:
!git remote add origin https://github.com/KonradWasikiewicz/dw_matrix_car.git

fatal: remote origin already exists.


In [74]:
!git push --all origin

Everything up-to-date
